## Artificial Intelligence Example
In this notebook, we will go through an example using a Neural Network called Squeezenet. Squeezenet is a Neural Net based off of the Alexnet model. We use it in this example because it is lightweight and fast.

In the following cell, we download a pretrained Squeezenet model from the PyTorch API. PyTorch is an ML framework that can be used to build neural nets. It also maintains a zoo of pretrained models that can be downloaded and used with little effort.

In [ ]:
import requests, torch, torchvision
from torchvision import models, transforms
squeezenet_model = models.squeezenet1_1(pretrained=True)

Now we define the transformations that we will do on the inputs before passing them to the model. We will also download labels to use.

In [ ]:
# First we define the preproccessing on the images:
normalize = transforms.Normalize(
   mean=[0.485, 0.456, 0.406],
   std=[0.229, 0.224, 0.225]
)
preprocess = transforms.Compose([
   transforms.Resize(256),
   transforms.CenterCrop(224),
   transforms.ToTensor(),
   normalize
])

# Then we download the labels:
labels = {int(key):value for (key, value)
          in requests.get('https://s3.amazonaws.com/outcome-blog/imagenet/labels.json').json().items()}

Now we define a predict function. Its input is an array of strings - either URL's or filenames - of images to run the model on. It returns an array of labels corresponding to each image.

In [ ]:
def predict_torch_model(imgs, download=False):
    import io
    import PIL.Image
    from io import BytesIO

    # We first prepare a batch from `imgs`
    img_tensors = []
    for img in imgs:
        if not download:
            img_tensor = preprocess(PIL.Image.open(img))
        else:
            response = requests.get(img)
            img_tensor = preprocess(PIL.Image.open(BytesIO(response.content)))
        img_tensor.unsqueeze_(0)
        img_tensors.append(img_tensor)
    img_batch = torch.cat(img_tensors)
    
    # We perform a forward pass
    with torch.no_grad():
        model_output = squeezenet_model(img_batch)
        
    # Parse Result
    img_labels = [labels[out.data.numpy().argmax()] for out in model_output]
        
    return img_labels

First we run it with some predownloaded files:

In [ ]:
# Blue is our cat's name. The last picture is an image of our cat.
images = ['images/blue.JPG', 'images/cat.jpg', 'images/duck.jpg']
from IPython.display import display, Image
for img in images:
    print('Output for', img)
    display(Image(img, width=200, height=200))
    print(predict_torch_model([img]))

Now we run it with some files we download from the web. We also note that AI can fail - while we got pretty good labels for the above images, we will see some test cases below where our results aren't as good.

In [ ]:
# Blue is our cat's name. The last picture is an image of our cat.
download_images = ['https://techcrunch.com/wp-content/uploads/2015/02/shutterstock_175625024.jpg',
                   'https://www.emmys.com/sites/default/files/styles/show_detail/public/disney-mickey-mouse-600x600.jpg']
from IPython.display import display, Image
for img in download_images:
    print('Output for', img)
    display(Image(img, width=200, height=200))
    print(predict_torch_model([img], download=True))